# Imputation Methods

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
import eda_tools as et
import impute_eval as ie
import data_munging_tools as dmt
import model_fitting_tools as mft

In [4]:
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, SimpleFill, MICE, MatrixFactorization, IterativeSVD

In [5]:
#instantiate imputers:
sf_median = SimpleFill(fill_method="median")
sf_mean = SimpleFill(fill_method="mean")
knn_imputer = KNN(k=10, verbose=0)
mice_imputer = MICE(verbose=0)
mf_imputer = MatrixFactorization(verbose=0)
nnm_imputer = NuclearNormMinimization(verbose=0)
soft_imputer = SoftImpute(verbose=0)
itersvd_imputer = IterativeSVD(verbose=0)
nonnormed_imputers_dict = {"sf_median" : sf_median, "sf_mean" : sf_mean, "knn_imputer" : knn_imputer}
imputers_dict = {"sf_median" : sf_median, "sf_mean" : sf_mean, "knn_imputer" : knn_imputer, "mice_imputer" : mice_imputer, "soft_imputer": soft_imputer}
all_imputers_dict = {"sf_median" : sf_median, "sf_mean" : sf_mean, "knn_imputer" : knn_imputer, "mice_imputer": mice_imputer, "mf_imputer": mf_imputer}

In [6]:
pd.set_option('max_colwidth', 800)

import matplotlib.pyplot as plt
%matplotlib inline

## Step 0: Data Prep

In [7]:
testing_df = pd.read_csv('data/cleaned-input.test.tsv', sep='\t', low_memory=False)
training_df = pd.read_csv('data/cleaned-input.training.tsv', sep='\t', low_memory=False)

In [8]:
merged_df = pd.concat([testing_df, training_df])

In [9]:
my_blacklist_patterns = ['^recent_ipt_', '^production_', 'total_num_stages', 'bakken_isopach_ft']

In [10]:
df = merged_df.copy()
print df.shape
df.drop(["FileNo", "Section"], axis=1, inplace=True)
print df.shape

(8115, 53)
(8115, 51)


In [11]:
target = 'production_liquid_90'

In [12]:
df = dmt.munge_pipe(df, blacklist_patterns=my_blacklist_patterns, exceptions=set([target]), null_cutoff=.2)

df shape before removals (8115, 51)
Shape before blacklist removal: (8115, 51)
Blacklisted columns: ['bakken_isopach_ft', 'production_liquid_120', 'production_liquid_150', 'production_liquid_180', 'production_liquid_1825', 'production_liquid_270', 'production_liquid_30', 'production_liquid_365', 'production_liquid_60', 'production_liquid_730', 'total_num_stages']
Number of blacklisted columns: 11
Shape after blacklist removal: (8115, 40)
**************************************************
Shape before cardinality removal: (8115, 40)
Dropped CurrentWellName since it was categorical and had a high cardinality
Dropped Footages since it was categorical and had a high cardinality
Dropped LeaseName since it was categorical and had a high cardinality
Dropped LeaseNumber since it was categorical and had a high cardinality
Dropped OriginalWellName since it was categorical and had a high cardinality
Dropped api since it was categorical and had a high cardinality
Dropped spud_date since it was cat

## Step 1: Test a single imputation method on a single feature

In [13]:
impute_test_df = dmt.drop_nonnumeric_features(df)

Shape before removal: (8115, 31)
Columns dropped: ['CountyName', 'CurrentOperator', 'FieldName', 'OriginalOperator', 'ProducedPools', 'QQ', 'Range', 'Township', 'WellStatus', 'WellType', 'Wellbore', 'choke_size', 'stimulated_formation', 'type_treatment']
Shape after removal: (8115, 17)


In [14]:
impute_test_df.drop(target, axis=1, inplace=True)

In [16]:
impute_test_df.shape
impute_test_df.dropna(inplace=True)
impute_test_df.shape

(6113, 16)

In [17]:
ie.pandas_imputer_eval(impute_test_df, "total_lbs_proppant")

pandas fill_na(mean) mae, rmse:  1003134.60457 1955520.80819
pandas fill_na(median) mae, rmse:  977474.503306 1954656.34914


#### Just proving that simple imputer is the same as pandas

In [18]:
ie.fancy_imputer_eval(impute_test_df, "total_lbs_proppant", sf_mean)

{'mae': 1003134.6045664609, 'rmse': 1955520.8081882647}

In [19]:
ie.fancy_imputer_eval(impute_test_df, "total_lbs_proppant", sf_median)

{'mae': 977474.50330578513, 'rmse': 1954656.3491413717}


#### KNN

In [20]:
ie.fancy_imputer_eval(impute_test_df, "total_lbs_proppant", knn_imputer)

{'mae': 621423.97947035055, 'rmse': 1622344.6518944211}

In [21]:
#using subset of columns
knn_columns = ["total_lbs_proppant", "total_volume_bbls", "tvd", "GRElev", "legs"]

In [22]:
#Not so hot
ie.fancy_imputer_eval(impute_test_df[knn_columns], "total_lbs_proppant", knn_imputer)

{'mae': 869433.76528520964, 'rmse': 1844774.9972242971}

#### MICE, Soft Impute, Iterative SVD

In [23]:
ie.fancy_imputer_eval(impute_test_df, "total_lbs_proppant", mice_imputer)

{'mae': 905228.99666581035, 'rmse': 1875489.7876750953}

In [24]:
ie.fancy_imputer_eval(impute_test_df, "total_lbs_proppant", soft_imputer)

{'mae': 853836.30141906033, 'rmse': 1831365.0489999454}

In [29]:
ie.fancy_imputer_eval(impute_test_df, "total_lbs_proppant", itersvd_imputer)

{'mae': 984506.84450548701, 'rmse': 1952488.8428959888}

#### Matrix Factorization

In [26]:
# %time function doesn't work. only times first iteration
mf_result = ie.fancy_imputer_eval(impute_test_df, "total_lbs_proppant", mf_imputer)

train: 1 of 1 mini-batches from (6113, 16)
downhill: compiling evaluation function
downhill: compiling Adam optimizer
downhill: setting: rms_halflife = 14
downhill: setting: rms_regularizer = 1e-08
downhill: setting: patience = 5
downhill: setting: validate_every = 10
downhill: setting: min_improvement = 0.005
downhill: setting: max_gradient_norm = 5
downhill: setting: max_gradient_elem = 0
downhill: setting: learning_rate = TensorConstant{0.001}
downhill: setting: momentum = 0
downhill: setting: nesterov = False
downhill: validation 0 loss=10.585778 error=10.498290 *
downhill: Adam 1 loss=10.585778 error=10.498290
downhill: Adam 2 loss=10.550742 error=10.463365
downhill: Adam 3 loss=10.503588 error=10.416360
downhill: Adam 4 loss=10.448639 error=10.361585
downhill: Adam 5 loss=10.388262 error=10.301401
downhill: Adam 6 loss=10.324013 error=10.237356
downhill: Adam 7 loss=10.257000 error=10.170558
downhill: Adam 8 loss=10.188053 error=10.101833
downhill: Adam 9 loss=10.117810 error=10.

In [27]:
mf_result

{'mae': 887144.37798303901, 'rmse': 1922065.7263672766}

In [28]:
# Throws error. missing an argument?
ie.fancy_imputer_eval(impute_test_df, "total_lbs_proppant", nnm_imputer)

KeyboardInterrupt: 

## Step 2: Test on all numeric features

#### Test on all features, one at a time.
*Pseudocode*:

```python
for each (numeric) column in dataframe: 
    replace 10% of the observations in one column with None  
    for each imputation method in dict:
        fill missing values
        compute error score for each combination of feature/imputation method
rollup results into a df
    ```

#### Adapted from code in documentation for fancyimpute library  https://pypi.python.org/pypi/fancyimpute

In [83]:
impute_results_df = ie.imputers_eval(impute_test_df, imputers_dict)

now imputing values for column GRElev with imputer soft_imputer
now imputing values for column GRElev with imputer sf_median
now imputing values for column GRElev with imputer mice_imputer
now imputing values for column GRElev with imputer sf_mean
now imputing values for column GRElev with imputer knn_imputer
now imputing values for column KBElev with imputer soft_imputer
now imputing values for column KBElev with imputer sf_median
now imputing values for column KBElev with imputer mice_imputer
now imputing values for column KBElev with imputer sf_mean
now imputing values for column KBElev with imputer knn_imputer
now imputing values for column TD with imputer soft_imputer
now imputing values for column TD with imputer sf_median
now imputing values for column TD with imputer mice_imputer
now imputing values for column TD with imputer sf_mean
now imputing values for column TD with imputer knn_imputer
now imputing values for column bh_lat with imputer soft_imputer
now imputing values for

In [84]:
impute_results_df

,knn_imputer,mice_imputer,sf_mean,sf_median,soft_imputer
GRElev,32.937391,43.859176,131.739782,132.000000,53.662910
KBElev,33.656877,47.500630,133.791984,132.403685,56.721286
TD,731.560670,1057.801104,1232.811161,1125.918760,924.276236
bh_lat,0.144739,0.145272,0.343438,0.343322,0.202749
bh_lng,0.150667,0.125485,0.394108,0.393342,0.177855
legs,0.337165,0.814652,0.351858,0.219430,0.693342
max_tvd,74.487098,62.991736,612.034433,574.316759,84.225039
mean_tvd,77.304638,69.511239,619.554369,581.414754,96.100281
min_tvd,84.795253,135.695898,632.089117,596.919481,164.440233
num_pools_produced,0.159680,0.573178,0.000366,0.000000,0.475580


## Step 3: Check each imputer at different proportions of missing values

In [126]:
knn_impute_test = ie.imputers_percent_eval(impute_test_df, knn_imputer, impute_percents = range(10, 71, 10))

computing error for filling column: GRElev
computing error for filling column: KBElev
computing error for filling column: TD
computing error for filling column: bh_lat
computing error for filling column: bh_lng
computing error for filling column: legs
computing error for filling column: max_tvd
computing error for filling column: mean_tvd
computing error for filling column: min_tvd
computing error for filling column: num_pools_produced
computing error for filling column: std_tvd
computing error for filling column: surface_lat
computing error for filling column: surface_lng
computing error for filling column: total_lbs_proppant
computing error for filling column: total_volume_bbls
computing error for filling column: tvd


In [131]:
mice_impute_test = ie.imputers_percent_eval(impute_test_df, mice_imputer, impute_percents = range(10, 71, 10))

computing error for filling column: GRElev
computing error for filling column: KBElev
computing error for filling column: TD
computing error for filling column: bh_lat
computing error for filling column: bh_lng
computing error for filling column: legs
computing error for filling column: max_tvd
computing error for filling column: mean_tvd
computing error for filling column: min_tvd
computing error for filling column: num_pools_produced
computing error for filling column: std_tvd
computing error for filling column: surface_lat
computing error for filling column: surface_lng
computing error for filling column: total_lbs_proppant
computing error for filling column: total_volume_bbls
computing error for filling column: tvd


In [132]:
soft_impute_test = ie.imputers_percent_eval(impute_test_df, soft_imputer, impute_percents = range(10, 71, 10))

computing error for filling column: GRElev
computing error for filling column: KBElev
computing error for filling column: TD
computing error for filling column: bh_lat
computing error for filling column: bh_lng
computing error for filling column: legs
computing error for filling column: max_tvd
computing error for filling column: mean_tvd
computing error for filling column: min_tvd
computing error for filling column: num_pools_produced
computing error for filling column: std_tvd
computing error for filling column: surface_lat
computing error for filling column: surface_lng
computing error for filling column: total_lbs_proppant
computing error for filling column: total_volume_bbls
computing error for filling column: tvd


In [133]:
mice_impute_test

,10,20,30,40,50,60,70
GRElev,43.859176,47.292672,59.909084,59.729627,75.589308,74.440601,78.867309
KBElev,47.500630,51.509411,60.822847,62.552212,72.295283,73.082798,77.132953
TD,1057.801104,1036.720348,1038.295411,1048.888312,1034.768220,1047.915393,1039.206656
bh_lat,0.145272,0.154930,0.166705,0.178471,0.193264,0.214844,0.242051
bh_lng,0.125485,0.132641,0.144727,0.154933,0.170531,0.189089,0.218686
legs,0.814652,0.799094,0.838261,0.881418,0.928876,0.987293,1.043671
max_tvd,62.991736,62.463248,62.755925,63.655153,62.812946,63.144791,62.294902
mean_tvd,69.511239,70.508470,71.397284,71.402724,70.585173,71.256478,70.602198
min_tvd,135.695898,138.108530,138.204045,140.157413,138.114178,138.614207,137.669464
num_pools_produced,0.573178,0.569356,0.584489,0.599762,0.604379,0.615715,0.590880


In [134]:
soft_impute_test

,10,20,30,40,50,60,70
GRElev,53.662910,60.101196,75.039792,76.488958,96.819509,98.381316,107.502043
KBElev,56.721286,63.234985,73.215926,75.978631,86.914250,89.570082,96.047288
TD,924.276236,905.342988,904.621032,913.504730,899.793599,911.519611,904.016817
bh_lat,0.202749,0.251365,0.298682,0.351815,0.417336,0.510591,0.641181
bh_lng,0.177855,0.219036,0.265541,0.313749,0.378710,0.462107,0.597047
legs,0.693342,0.668174,0.693918,0.726686,0.766882,0.817012,0.881650
max_tvd,84.225039,96.001773,105.872527,119.340362,127.495856,142.744705,166.822836
mean_tvd,96.100281,108.202927,118.261842,130.328760,137.921487,153.001467,176.820877
min_tvd,164.440233,177.045295,180.836927,190.494253,193.773154,205.496777,224.428765
num_pools_produced,0.475580,0.482245,0.497710,0.521565,0.557318,0.611382,0.669655


In [124]:
median_impute_test = ie.imputers_percent_eval(impute_test_df, sf_median, impute_percents = range(10, 71, 10))

computing error for filling column: GRElev
computing error for filling column: KBElev
computing error for filling column: TD
computing error for filling column: bh_lat
computing error for filling column: bh_lng
computing error for filling column: legs
computing error for filling column: max_tvd
computing error for filling column: mean_tvd
computing error for filling column: min_tvd
computing error for filling column: num_pools_produced
computing error for filling column: std_tvd
computing error for filling column: surface_lat
computing error for filling column: surface_lng
computing error for filling column: total_lbs_proppant
computing error for filling column: total_volume_bbls
computing error for filling column: tvd


In [125]:
median_impute_test

,10,20,30,40,50,60,70
GRElev,132.000000,137.066556,149.398880,147.864671,164.645455,1.604661e+02,1.622120e+02
KBElev,132.403685,136.673045,137.715966,138.477234,138.171380,1.378322e+02,1.378958e+02
TD,1125.918760,1104.428453,1108.306723,1122.128162,1104.415152,1.119278e+03,1.111232e+03
bh_lat,0.343322,0.351014,0.350008,0.351345,0.347306,3.504301e-01,3.482546e-01
bh_lng,0.393342,0.381522,0.379197,0.379873,0.380585,3.828222e-01,3.845364e-01
legs,0.219430,0.202995,0.204482,0.206577,0.202694,2.050633e-01,2.059595e-01
max_tvd,574.316759,569.065408,567.167289,568.935919,554.926283,5.583169e+02,5.558980e+02
mean_tvd,581.414754,578.330105,577.214216,578.200146,564.057762,5.670143e+02,5.648621e+02
min_tvd,596.919481,598.801897,595.914532,599.123988,588.276572,5.910490e+02,5.885211e+02
num_pools_produced,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00


In [122]:
mean_impute_test = ie.imputers_percent_eval(impute_test_df, sf_mean, impute_percents = range(10, 71, 10))

computing error for filling column: GRElev
computing error for filling column: KBElev
computing error for filling column: TD
computing error for filling column: bh_lat
computing error for filling column: bh_lng
computing error for filling column: legs
computing error for filling column: max_tvd
computing error for filling column: mean_tvd
computing error for filling column: min_tvd
computing error for filling column: num_pools_produced
computing error for filling column: std_tvd
computing error for filling column: surface_lat
computing error for filling column: surface_lng
computing error for filling column: total_lbs_proppant
computing error for filling column: total_volume_bbls
computing error for filling column: tvd


In [168]:
mean_imp = mean_impute_test.unstack().reset_index()

In [170]:
mean_imp.columns=["perc_imputed", "feature", "mae"]

In [172]:
mean_imp["imputer"] = "Mean"

In [173]:
mean_imp

,perc_imputed,feature,mae,imputer
0,10,GRElev,1.317398e+02,Mean
1,10,KBElev,1.337920e+02,Mean
2,10,TD,1.232811e+03,Mean
3,10,bh_lat,3.434379e-01,Mean
4,10,bh_lng,3.941079e-01,Mean
5,10,legs,3.518579e-01,Mean
6,10,max_tvd,6.120344e+02,Mean
7,10,mean_tvd,6.195544e+02,Mean
8,10,min_tvd,6.320891e+02,Mean
9,10,num_pools_produced,3.664346e-04,Mean


In [180]:
soft_imp = soft_impute_test.unstack().reset_index()
soft_imp.columns=["perc_imputed", "feature", "mae"]
soft_imp["imputer"] = "SoftMax"
soft_imp

,perc_imputed,feature,mae,imputer
0,10,GRElev,53.662910,SoftMax
1,10,KBElev,56.721286,SoftMax
2,10,TD,924.276236,SoftMax
3,10,bh_lat,0.202749,SoftMax
4,10,bh_lng,0.177855,SoftMax
5,10,legs,0.693342,SoftMax
6,10,max_tvd,84.225039,SoftMax
7,10,mean_tvd,96.100281,SoftMax
8,10,min_tvd,164.440233,SoftMax
9,10,num_pools_produced,0.475580,SoftMax


In [199]:
mice_imp = mice_impute_test.unstack().reset_index()
mice_imp.columns=["perc_imputed", "feature", "mae"]
mice_imp["imputer"] = "MICE"
mice_imp.shape

(112, 4)

In [184]:
median_imp = median_impute_test.unstack().reset_index()
median_imp.columns=["perc_imputed", "feature", "mae"]
median_imp["imputer"] = "Median"
median_imp

,perc_imputed,feature,mae,imputer
0,10,GRElev,1.320000e+02,Median
1,10,KBElev,1.324037e+02,Median
2,10,TD,1.125919e+03,Median
3,10,bh_lat,3.433218e-01,Median
4,10,bh_lng,3.933416e-01,Median
5,10,legs,2.194305e-01,Median
6,10,max_tvd,5.743168e+02,Median
7,10,mean_tvd,5.814148e+02,Median
8,10,min_tvd,5.969195e+02,Median
9,10,num_pools_produced,0.000000e+00,Median


In [196]:
percent_imputation_results = pd.concat([mean_imp, median_imp, knn_imp, mice_imp, soft_imp]).reset_index(drop=True)

In [198]:
percent_imputation_results.to_csv("percent_imputation_results.csv")

In [197]:
percent_imputation_results

,perc_imputed,feature,mae,imputer
0,10,GRElev,1.317398e+02,Mean
1,10,KBElev,1.337920e+02,Mean
2,10,TD,1.232811e+03,Mean
3,10,bh_lat,3.434379e-01,Mean
4,10,bh_lng,3.941079e-01,Mean
5,10,legs,3.518579e-01,Mean
6,10,max_tvd,6.120344e+02,Mean
7,10,mean_tvd,6.195544e+02,Mean
8,10,min_tvd,6.320891e+02,Mean
9,10,num_pools_produced,3.664346e-04,Mean


In [70]:
percent_imputation_results = pd.read_csv("percent_imputation_results.csv")

In [71]:
percent_imputation_results.drop("Unnamed: 0", axis=1, inplace=True)

In [72]:
percent_imputation_results.head()

,perc_imputed,feature,mae,imputer
0,10,GRElev,131.739782,Mean
1,10,KBElev,133.791984,Mean
2,10,TD,1232.811161,Mean
3,10,bh_lat,0.343438,Mean
4,10,bh_lng,0.394108,Mean


In [79]:
percent_imputation_results['imputer'].replace("SoftMax", "Soft Impute",inplace=True)

In [80]:
percent_imputation_results.imputer.value_counts()

Soft Impute    112
Mean           112
MICE           112
Median         112
KNN            112
Name: imputer, dtype: int64

In [42]:
percent_imputation_results[(percent_imputation_results["imputer"] == "KNN") & (percent_imputation_results["feature"] == "GRElev")]["mae"]

224    32.937391
240    36.085411
256    49.428849
272    49.671205
288    68.243090
304    66.453972
320    71.387988
Name: mae, dtype: float64

# Plotting

### Font size issues

In [81]:
import plotly.plotly as py
import plotly.graph_objs as go

In [96]:
def plot_errors(feature):
    current = feature
    N = 10
    x_axis = np.linspace(10, 100, 10)
#     svd_y = svd_impute_test.loc[current].values
    knn_y = percent_imputation_results[(percent_imputation_results["imputer"] == "KNN") & (percent_imputation_results["feature"] == current)]["mae"]
    mice_y = percent_imputation_results[(percent_imputation_results["imputer"] == "MICE") & (percent_imputation_results["feature"] == current)]["mae"]
    soft_y = percent_imputation_results[(percent_imputation_results["imputer"] == "Soft Impute") & (percent_imputation_results["feature"] == current)]["mae"]
    mean_y = percent_imputation_results[(percent_imputation_results["imputer"] == "Mean") & (percent_imputation_results["feature"] == current)]["mae"]
    median_y = percent_imputation_results[(percent_imputation_results["imputer"] == "Median") & (percent_imputation_results["feature"] == current)]["mae"]

    # Create traces
    trace0 = go.Scatter(
        x =  x_axis,
        y = knn_y,
        mode = 'lines+markers',
        name = 'KNN Imputer'
    )
    trace1 = go.Scatter(
        x =  x_axis,
        y = median_y,
        mode = 'lines+markers',
        name = 'Median Imputer'
    )

    trace2 = go.Scatter(
        x = x_axis,
        y = mean_y,
        mode = 'lines+markers',
        name = 'Mean Imputer'
    )
    trace3 = go.Scatter(
        x = x_axis,
        y = mice_y,
        mode = 'lines+markers',
        name = 'MICE Imputer'
    )
    trace4 = go.Scatter(
        x = x_axis,
        y = soft_y,
        mode = 'lines+markers',
        name = 'Softmax Imputer'
    )
    


    data = [trace0, trace1, trace4]


    layout = dict(title = current,
                  xaxis = dict(title = 'Percent of data imputed'),
                 yaxis = dict(title = 'Mean Absolute Error'),
                  )

    fig = dict(data=data, layout=layout)

    return py.iplot(fig, filename='line-mode')


## Some top features:
### 'surface_lat', 'bh_lat', 'total_lbs_proppant', 'total_volume_bbls', 'mean_tvd', 'max_tvd'

In [98]:
plot_errors("surface_lat")

In [88]:
plot_errors("total_lbs_proppant")

In [89]:
plot_errors("total_volume_bbls")

In [144]:
plot_errors("mean_tvd")

In [99]:
plot_errors("legs")